# Correlation vs Linear regression

- Correlation (-1 to 1) is used to find <u>strength and direction</u> of linear relationship between two variables

- In Simple linear regression we <u>quantify and model</u> this linear relationship
  - By fitting a straight line to data points
  

# The LR Model

Assignment: The VP of Engagement asks you to build the model describing the relationship between a DataSciencester user’s <u>number of friends and the amount of time the user spends</u> on the site each day.

- we are convinced that its a linear relationship looking at correlation (0.587) - see C5 - Statistics
- therefore, let's start with a linear model

## Linear model hypothesis

\begin{equation}
y_i = \beta x_i + \alpha + \epsilon_i
\tag{1}
\end{equation}


>$y_i$ is the number of minutes user i spends on the site daily  
>$x_i$ is the number of friends user i has  
>$ε_i$ is a (hopefully small) error term representing the fact that there are other factors not accounted for by this simple model

## Steps to train the simple LR model

### 1. Using normal equations (No iterations) - Least squares fit

Find optimized $\alpha$ and $\beta$ from given x,y using formula:
   \begin{equation}
   \beta = \frac{r \sigma_y}{\sigma_x}
   \tag{2}
   \end{equation}
   >$r$ = correlation(x,y)  
   >$\sigma_y$ = Standard deviation of y  
   >$\sigma_x$ = Standard deviation of x

   \begin{equation}
   \alpha =  \bar{y} - \beta \bar{x}
   \tag{3}
   \end{equation}
   > derived from equation (1)

- Used when only one predictor ($x$) is present
- Its a quick solution
- Lower accuracy
- Error is not minimized iteratively

### 2. Using error minimization through Gradient Descent

1. Predict $y_i$ from $ \alpha , \beta, x_i$ using,
   \begin{equation}
   y_{predicted} = \beta x_i + \alpha
   \tag{4}
   \end{equation}

2. Calculate error between $y_{predicted}$ and given $y_{i}$ and calculate <u>sum of squared errors</u> (because + and - errors may cancel out each other)
   \begin{equation}
   error = y_{predicted} - y_{i}
   \tag{5}
   \end{equation}

4. Find R-squared (Coefficient of determination) which measures the fraction of the total variation in the dependent variable that is captured by the model
5. Use **gradient descent** - minimize loss function ($error^2$) and get optimum values of $\alpha$ and $\beta$ for linear regression model

# Using normal equations - Least Squares Fit

(a) **Using calculus the optimized $\alpha$ and $\beta$ are given by using equation 2 and 3**

In [4]:
from typing import List, Tuple
from scratch.statistics import correlation, standard_deviation, mean

def least_squares_fit(x: List[float], y: List[float]) -> Tuple[float, float]:
    """
    Given two vectors x and y,
    find the least-squares values of alpha and beta
    """
    beta = correlation(x,y) * standard_deviation(y) /standard_deviation(x)
    alpha = mean(y) - beta * mean(x)
    return alpha, beta

Let's think about why this might be a reasonable solution. 

> * The choice of $\alpha$ simply says that -  when we see the average value of the independent variable x, we predict the average value of the dependent variable y.  
> * The choice of $\beta$ means that - when the input value increases by standard_deviation(x), the prediction then increases by `correlation(x, y) * standard_deviation(y)`.  
> In the case where x and y are perfectly correlated, a one-standard-deviation increase in x results in a one-standard-deviation-of-y increase in the prediction.    
> * When they’re perfectly anticorrelated, the increase in x results in a decrease in the prediction.
> * And when the correlation is 0, beta is 0, which means that changes in x don’t affect the prediction at all

In [10]:
# quick test

# get a sample data
x = [i for i in range(-100, 110, 10)]
y = [3 * i - 5 for i in x]

alpha, beta = least_squares_fit(x, y) 
assert -5.2 < alpha < -4.8
assert 2.8 < beta < 3.2


In [11]:
# Let's check with outliers

# generate data with outlier
import matplotlib.pyplot as plt
import random
from scratch.statistics import num_friends_good, daily_minutes_good

# Now check what's the alpha and beta with outlier data
alpha, beta = least_squares_fit(num_friends_good, daily_minutes_good)

print(alpha, beta)

22.752546656812427 0.9322227999047837


> - This gives values of alpha = 22.95 and beta = 0.903.
>   
> - So our model says that we expect a user with n friends to spend 22.95 + n * 0.903 minutes on the site each day.
>     
> - That is, we predict that a user with no friends on DataSciencester would still spend about 23 minutes a day on the site.
>   
> - And for each additional friend, we expect a user to spend almost a minute more on the site each day.  



**Effect of dataset size on values of $\alpha$ and $\beta$:**
> - For large number of samples the values of alpha and beta will be optimized to real values
> 
> - But for low number of samples it will be highly deviating
>   
> - You can run above for less number of data in dataset and see how the alpha and beta are deviated from real values.

## R-squared/ Coefficient of Determination

-  It is a commonly used metric <u>to evaluate the goodness of fit of a regression model</u>.

-  Higher values of $R^2$ indicate a better fit, while lower values indicate a poorer fit.

- However, $R^2$ alone may not provide a complete picture of the model's performance, so it's often used in conjunction with other metrics for model evaluation.

- It is given by:
  $$R^2 =  1 - \frac{\sum{(y_p - y)^2}}{\sum{(y - \bar{y})^2}} = 1 - \frac{\text{sum of sqerrors}(alpha, beta, x, y)}{\text{total sum of squares}(y)}$$


**Predict $y$** using $\alpha , \beta , x$

- Assuming we have determined \alpha and \beta, we can make predictions as following:

In [12]:
def predict(alpha: float, beta: float, x_i: float) -> float:
    return beta * x_i + alpha

**Calculate error** from known y value and predicted y value

- how do we calculate $\alpha$ and $\beta$ here?
 > Well, any choice of alpha and beta gives us a predicted output for each input x_i

In [13]:
def error(alpha: float, beta: float, x_i: float, y_i: float) -> float:
    """
    The error from predicting beta * x_i + alpha
    when the actual value is y_i
    """
    return predict(alpha, beta, x_i) - y_i

What we’d really like to know is the **total error over the entire dataset**. But we don’t want to just add the errors—if the prediction for x_1 is too high and the prediction for x_2 is too low, the errors may just cancel out.
So instead **we add up the squared errors**:

In [14]:
def sum_of_sqerrors(alpha: float, beta: float, x: List[float], y: List[float]) -> float:
    return sum((error(alpha, beta, x_i, y_i) ** 2) for x_i, y_i in zip(x,y))

In [16]:
from scratch.statistics import de_mean

def total_sum_of_squares(y: List[float]) -> float:
    """
    the total squared variation of y_i's from their mean
    """
    return sum(v ** 2 for v in de_mean(y))
    
def r_squared(alpha: float, beta: float, x: List[float], y: List[float]) -> float:
    """
    the fraction of variation in y captured by the model, which equals
    1 - the fraction of variation in y not captured by the model
    """
    return 1.0 - (sum_of_sqerrors(alpha, beta, x, y) / total_sum_of_squares(y))

r = r_squared(alpha, beta, num_friends_good, daily_minutes_good)
assert 0.328 < r < 0.329

- for optimized $\alpha$ and $\beta$, sum of square errors -> 0

- thus $R^2$ -> 1

- higher the $R^2$, better the model fits

- Here we calculate an R-squared of 0.328, which tells us that our model is only sort of okay at fitting the data, and that <u>clearly there are other factors at play.</u>

# Using Gradient Descent - minimize loss function

Steps:
1. Select random $\alpha$ and $\beta$ and a learning rate
2. Minimize loss function for optimized $\alpha$ and $\beta$
3. For this - find gradient of loss function wrt $\alpha$ and $\beta$
4. Update gradient step size using function gradient_step from scratch.gradient_descent
5. Iterate and optimize $\alpha$, $\beta$



Let's say, $$ \theta = [\alpha, \beta]$$

In [ ]:
import random
from scratch.gradient_descent import gradient_step
import tqdm

num_epochs = 100000
random.seed(0)
guess = [random.random(), random.random()] # choose random value to start

print(f"{guess=}")
learning_rate = 0.00001 # set step size

with tqdm.trange(num_epochs) as t:
    for i in t:
        alpha, beta = guess
        # print(alpha,beta)
        
        # Partial derivate of loss function (error^2) wrt alpha
        grad_alpha = sum(2 * error(alpha, beta, x_i, y_i)
                 for x_i, y_i in zip(num_friends_good,daily_minutes_good))

        grad_beta = sum(2 * error(alpha, beta, x_i, y_i) * x_i
                        for x_i, y_i in zip(num_friends_good,daily_minutes_good))

        # Compute loss to stick in the tqdm description
        loss = sum_of_sqerrors(alpha, beta,num_friends_good,daily_minutes_good)
        if i % 10 == 0:
            t.set_description(f"alpha: {alpha:.3f}, beta: {beta:.3f}, loss: {loss:.3f}")
        
        guess = gradient_step(guess, [grad_alpha, grad_beta], -learning_rate)



# We should get same result
alpha, beta = guess
assert 22.94 < alpha < 22.95
assert 0.75 < beta < 0.76

In [43]:
# Let's train and test some random data

X = [i for i in range(500)]
Y = [3 * i - 5 for i in X]

In [44]:
from scratch.machine_learning import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, 0.7)

In [62]:
import random
from scratch.gradient_descent import gradient_step
import tqdm

random.seed(0)
theta = [-4, 2] # choose random value to start

print(theta)
learning_rate = 0.00000008 # set step size

with tqdm.trange(100000) as t:
    for i in t:
        alpha, beta = theta
        # print(alpha,beta)
        
        # Partial derivate of loss function (error^2) wrt alpha
        grad_alpha = sum(2 * error(alpha, beta, x_i, y_i)
                 for x_i, y_i in zip(x_train,y_train))

        grad_beta = sum(2 * error(alpha, beta, x_i, y_i) * x_i
                        for x_i, y_i in zip(x_train,y_train))

        # Compute loss to stick in the tqdm description
        loss = sum_of_sqerrors(alpha, beta,x_train,y_train)
        if i % 100 == 0:
            t.set_description(f"alpha: {alpha:.3f}, beta: {beta:.3f}, loss: {loss:.3f}")
        
        theta = gradient_step(theta, [grad_alpha, grad_beta], -learning_rate)

alpha, beta = theta

[-4, 2]


alpha: -4.458, beta: 2.998, loss: 11.316: 100%|█| 100000/100000 [00:06<00:00, 16


In [63]:
print(r_squared(alpha, beta, x_test, y_test))

0.9999996159390275


# Maximum Likelihood Estimation

- Imagine you have a bunch of points on a graph, like dots scattered around.
  
- You want to draw a straight line that goes as close as possible to all these points.
  
- Now, let's say you have two numbers: one for how steep the line is ($\beta$) and another for where the line crosses the y-axis ($\alpha$).
  
- The least squares method is a way to find the best ($\alpha$) and ($\beta$) for your line.

**But why do we choose the least squares method?**

- Well, imagine you have a bunch of guesses for $\alpha$ and $\beta$.
  
- For each guess, you draw a line and see how far away each point is from the line.
  
- You square those distances, add them all up, and that's your total "error" for that guess.

- The least squares method says, "Let's find the $\alpha$ and $\beta$ that make this total 'error' as small as possible."

- In other words, we're trying to minimize the sum of the squared distances from each point to the line.

**The LSM is like maximum likelihood estimation (MLE), why?** 

- Imagine each point has a tiny cloud of possible positions around it, like a little blob. 

- We're saying, "What's the most likely position for the line that makes these blobs overlap the least?"

- That's where the least squares method comes in—it's all about finding the line that makes the observed points the most probable.

**So, in simple terms, we choose the least squares method because it's like finding the line that makes the points on our graph the most likely to be where they are. And it turns out, if the errors (the distances from each point to the line) are normally distributed, this method is exactly like maximizing the likelihood of seeing our data.**

When we talk about the "errors" in a regression model, we mean the differences between the actual values of the dependent variable (the points on our graph) and the values predicted by our regression line. These differences represent how far off our predictions are from the actual data points.

Now, imagine plotting all these errors on a graph. If the errors follow a normal distribution, it means they're kind of like a bell curve—most of the errors are small, and fewer errors are really big. This is a common assumption in regression analysis.

Now, when we use the least squares method to find the best-fitting line, what we're actually doing is finding the line that minimizes the sum of the squares of these errors. We're making the errors as small as possible.

Now, imagine flipping the problem around. Instead of trying to minimize the errors, we're trying to maximize the likelihood of seeing our data. In other words, we're asking, "What line would make it most likely for us to see these particular data points, given that the errors follow a normal distribution?"

It turns out that these two ways of looking at the problem—the least squares method and maximizing the likelihood of seeing our data—are actually equivalent, as long as the errors are normally distributed. In other words, if the errors follow that bell curve shape, then using the least squares method gives us the same result as if we were trying to maximize the likelihood of seeing our data.


